In [1]:
pip install torch tensorflow torchvision

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset=torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader=torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)



testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features=nn.Sequential(
        nn.Conv2d(3,6,5), 
        nn.ReLU(),    
            
        nn.MaxPool2d(2,2),
        nn.Conv2d(6, 16, 5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)    
            
        )
        
        self.classifier=nn.Sequential(
        nn.Linear(16*5*5, 120),
        nn.ReLU(),
        nn.Linear(120,84),
        nn.ReLU(),
        nn.Linear(84, 10))
        
        
    def  forward(self, x):
        x=self.features(x)
        x=x.view(-1, 16*5*5)
        x=self.classifier(x)
        return x

In [ ]:
net=Net()

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epoch=10
for epoch in range(num_epoch):
    running_loss=0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels =data
        optimizer.zero_grad()
        outputs=net(inputs)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

In [ ]:
correct=0
total=0
 
with torch.no_grad():
    for data in testloader:
        images, labels =data
        outputs=net(images)
        
        _, predicted =torch.max(outputs, 1)
        
        total +=labels.size(0)
        
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))       
    